In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

from pylab import rcParams
rcParams['figure.figsize'] = 18, 12

import warnings
warnings.filterwarnings('ignore')

### 1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)


In [2]:
data = pd.read_csv('heart.csv')
data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


### 2. Подготовьте датасет к обучению моделей:
a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [4]:
cols = data.select_dtypes('object').columns.to_list()
cols

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [5]:
data = pd.get_dummies(data, columns=cols)
data

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,...,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,...,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
914,68,144,193,1,141,3.4,1,0,1,1,...,0,0,0,1,0,1,0,0,1,0
915,57,130,131,0,115,1.2,1,0,1,1,...,0,0,0,1,0,0,1,0,1,0
916,57,130,236,0,174,0.0,1,1,0,0,...,0,0,1,0,0,1,0,0,1,0


### 3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.


In [6]:
X = data.loc[:, data.columns != 'HeartDisease']
y = data['HeartDisease']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)

### 4. Обучите модель логистической регрессии с параметрами по умолчанию.


In [42]:
LogReg = LogisticRegression(random_state=13).fit(X_train, y_train)

### 5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])


In [43]:
score_1 = cross_validate(LogReg, X_test, y_test, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])

In [44]:
score1df = pd.DataFrame(score_1)
score1df

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
0,0.015986,0.004001,0.684211,0.818182,0.692308,0.750000
1,0.011999,0.004014,0.842105,0.900000,0.818182,0.857143
2,0.012030,0.000000,0.736842,0.700000,0.777778,0.736842
3,0.011999,0.000000,0.947368,1.000000,0.909091,0.952381
4,0.012000,0.004001,0.888889,0.800000,1.000000,0.888889
5,0.013800,0.000000,0.888889,0.800000,1.000000,0.888889
6,0.016000,0.000000,0.944444,1.000000,0.909091,0.952381
7,0.011999,0.004000,0.888889,0.900000,0.900000,0.900000
8,0.012000,0.004000,0.722222,0.800000,0.727273,0.761905
9,0.016000,0.000000,1.000000,1.000000,1.000000,1.000000


### 6. Оптимизируйте 3-4 параметра модели:
a) Используйте GridSearchCV.


In [11]:
from sklearn.model_selection import GridSearchCV

In [45]:
LogReg1 = LogisticRegression(random_state=13).fit(X_train, y_train)

param_grid = {'C': [0.1,0.15,0.25,0.45,0.85], 
              'penalty': ['l1', 'l2']}

grid = GridSearchCV(LogReg1, param_grid, cv=10, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(random_state=13),
             param_grid={'C': [0.1, 0.15, 0.25, 0.45, 0.85],
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

b) Используйте RandomizedSearchCV.


In [24]:
from sklearn.model_selection import RandomizedSearchCV

In [50]:
LogReg2 = LogisticRegression(random_state=13).fit(X_train, y_train)

param_grid_1 = {'C': [0.01,0.15,0.6,1,2], 
                'class_weight': ['uniform', 'distance']}

grid_1 = RandomizedSearchCV(LogReg2, param_grid_1, cv=10, scoring='accuracy')

grid_1.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=LogisticRegression(random_state=13),
                   param_distributions={'C': [0.01, 0.15, 0.6, 1, 2],
                                        'class_weight': ['uniform',
                                                         'distance']},
                   scoring='accuracy')

d) Повторите п. 5 после каждого итогового изменения параметров.


In [47]:
score_2 = cross_validate(LogReg1, X_test, y_test, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])
score2df = pd.DataFrame(score_2)
score2df

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
0,0.016026,0.000000,0.684211,0.818182,0.692308,0.750000
1,0.012008,0.003993,0.842105,0.900000,0.818182,0.857143
2,0.012010,0.004001,0.736842,0.700000,0.777778,0.736842
3,0.011999,0.000000,0.947368,1.000000,0.909091,0.952381
4,0.016000,0.000000,0.888889,0.800000,1.000000,0.888889
5,0.012000,0.004001,0.888889,0.800000,1.000000,0.888889
6,0.015998,0.004002,0.944444,1.000000,0.909091,0.952381
7,0.011999,0.004000,0.888889,0.900000,0.900000,0.900000
8,0.012000,0.000000,0.722222,0.800000,0.727273,0.761905
9,0.012000,0.004000,1.000000,1.000000,1.000000,1.000000


In [51]:
score_3 = cross_validate(LogReg2, X_test, y_test, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])
score3df = pd.DataFrame(score_3)
score3df

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
0,0.015036,0.003000,0.684211,0.818182,0.692308,0.750000
1,0.014000,0.002001,0.842105,0.900000,0.818182,0.857143
2,0.012999,0.002001,0.736842,0.700000,0.777778,0.736842
3,0.013000,0.002001,0.947368,1.000000,0.909091,0.952381
4,0.012999,0.002001,0.888889,0.800000,1.000000,0.888889
5,0.012999,0.002001,0.888889,0.800000,1.000000,0.888889
6,0.013000,0.002001,0.944444,1.000000,0.909091,0.952381
7,0.013000,0.003000,0.888889,0.900000,0.900000,0.900000
8,0.013000,0.002001,0.722222,0.800000,0.727273,0.761905
9,0.011999,0.003000,1.000000,1.000000,1.000000,1.000000


### 7. Сформулируйте выводы по проделанной работе:
a) Сравните метрики построенных моделей.


они идентичны. видимо я что-то наворотил. и откровенно говоря я запутался зачем мы делали кросс-валидацию и к чему все это.